In [1]:
cd drive/MyDrive/

/content/drive/MyDrive


In [53]:
import pandas as pd
df_train = pd.read_csv('train_clean_removed_emoticons.csv')
df_validation = pd.read_csv('validation_clean_removed_emoticons.csv')

In [128]:
sentences_training = df_train['Review Text'].astype(str).str.lower().values
labels_training = df_train['Rating'].values

possible_labels = list(set(df_train['Rating']))
y = pd.get_dummies(possible_labels)

sentences_validation = df_validation['Review Text'].astype(str).str.lower().values.tolist()
labels_validation = df_validation['Rating'].values

In [129]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(sentences_training)

In [130]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
training_sequences = tokenizer.texts_to_sequences(sentences_training)
training_padded = pad_sequences(training_sequences, maxlen=100, padding='post', truncating='post')

In [90]:
from keras.layers import (
    Conv1D,
    Dense,
    Embedding,
    GlobalMaxPooling1D,
    Input,
    MaxPooling1D,
    # Bidirectional,LSTM
    # GlobalMaxPool1D
)

MAX_VOCAB_SIZE = 10000
word2idx = tokenizer.word_index
num_words = min(MAX_VOCAB_SIZE, len(word2idx) + 1)
EMBEDDING_DIM = 100
MAX_SEQUENCE_LENGTH = 100

embedding_layer = Embedding(
    num_words,
    EMBEDDING_DIM,
    # weights=[embedding_matrix],
    input_length=MAX_SEQUENCE_LENGTH,
    # trainable=False,
)

In [111]:
from keras.models import Model
from tensorflow.keras.optimizers import Adam

input_ = Input(shape=(MAX_SEQUENCE_LENGTH,))
x = embedding_layer(input_)
# --------------------------------------
x = Conv1D(128, 3, activation="relu")(x)
x = MaxPooling1D(3)(x)
x = Conv1D(128, 3, activation="relu")(x)
x = MaxPooling1D(3)(x)
x = Conv1D(128, 3, activation="relu")(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation="relu")(x)
output = Dense(len(y), activation="softmax")(x)
# --------------------------------------
# x = Bidirectional(LSTM(15, return_sequences=True))(x)
# x = GlobalMaxPool1D()(x)
# output = Dense(len(y), activation="softmax")(x)
model = Model(input_, output)
model.compile(
    loss="categorical_crossentropy",
    #loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.01),
    metrics=["accuracy"],
)

In [118]:
import numpy as np

BATCH_SIZE = 128
x_train = training_padded
y_train = pd.get_dummies(labels_training)

r = model.fit(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=30,
    validation_split=0.1
)

Epoch 1/30
128/128 [==============================] - 2s 19ms/step - loss: 0.4545 - accuracy: 0.8100 - val_loss: 1.5333 - val_accuracy: 0.5447
Epoch 2/30
128/128 [==============================] - 2s 17ms/step - loss: 0.4194 - accuracy: 0.8269 - val_loss: 1.6688 - val_accuracy: 0.5552
Epoch 3/30
128/128 [==============================] - 2s 17ms/step - loss: 0.3963 - accuracy: 0.8390 - val_loss: 1.4752 - val_accuracy: 0.5889
Epoch 4/30
128/128 [==============================] - 2s 17ms/step - loss: 0.4001 - accuracy: 0.8333 - val_loss: 1.6861 - val_accuracy: 0.5756
Epoch 5/30
128/128 [==============================] - 2s 17ms/step - loss: 0.3583 - accuracy: 0.8523 - val_loss: 1.8774 - val_accuracy: 0.5806
Epoch 6/30
128/128 [==============================] - 2s 17ms/step - loss: 0.3409 - accuracy: 0.8601 - val_loss: 1.8050 - val_accuracy: 0.5425
Epoch 7/30
128/128 [==============================] - 2s 17ms/step - loss: 0.3159 - accuracy: 0.8711 - val_loss: 2.0291 - val_accuracy: 0.5679

In [195]:
y_train_pred = model.predict(x_train)
y_train_pred = np.argmax(y_train_pred, axis=1)

In [203]:
from sklearn.metrics import f1_score
f1_score(labels_training, y_train_pred, average='micro')

0.041351515486114945

In [204]:
tokenizer_validation = Tokenizer(num_words=10000)
tokenizer_validation.fit_on_texts(sentences_validation)
validation_sequences = tokenizer_validation.texts_to_sequences(sentences_validation)
validation_padded = pad_sequences(validation_sequences, maxlen=100, padding='post', truncating='post')

y_valid_pred = model.predict(validation_padded)
y_valid_pred = np.argmax(y_valid_pred, axis=1)

f1_score(labels_validation, y_valid_pred, average='micro')

0.16445623342175067